# 3D Houses Project

##### A jupyter notebook where you can search for a Belgium address and it will create a 3D House model using Lidar.

### Enter Belgium Address

First you need to enter the Belgium address you wanted to search.

In [173]:
address = input("Enter the Belgium address: ") 

Enter the Belgium address: Distelstraat 42, 9000 Gent


### Importing The Necessary Libraries

In [248]:
import os
from osgeo import gdal
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

# set this so the graphs open internally
%matplotlib inline
import seaborn as sns

import rioxarray as rxr
import earthpy as et
import rasterio as rt

### Opening The Belgium Address API

Then it will search the coordinates of the address in the OpenAddress Belgium api. To do that, we need to open the api first and convert it to a dataframe. 

In [3]:
file = 'https://opendata.bosa.be/download/best/openaddress-bevlg.zip'

In [6]:
api = pd.read_csv(file)

C:\Users\Arlene\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (5,8,9,12,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Creating a new dataframe with only the address and coordinates

In [7]:
belgium = api[['EPSG:31370_x', 'EPSG:31370_y', 'EPSG:4326_lat', 'EPSG:4326_lon']].copy()

Creating an address column which will concatenate the address value to only 1 column

In [8]:
belgium['address'] = api['streetname_nl'] + " " + api['house_number'] + ", " + api['postcode'].apply(str) + " " + api['municipality_name_nl']

In [13]:
belgium

,EPSG:31370_x,EPSG:31370_y,EPSG:4326_lat,EPSG:4326_lon,address
0,190710.56,224659.47,51.330292,4.952837,"Steenweg op Oosthoven 51, 2300 Turnhout"
1,158832.72,192858.98,51.045828,4.494708,"Duivenstraat 102, 2800 Mechelen"
2,114333.83,206615.16,51.168450,3.858827,"Meersstraat 7, 9185 Wachtebeke"
3,101669.12,179169.21,50.920830,3.681393,"Nijverheidsstraat 67, 9890 Gavere"
4,103887.16,195997.07,51.072272,3.710818,"Klaverstraat 1A, 9000 Gent"
...,...,...,...,...,...
3867218,111243.39,173887.03,50.874065,3.818109,"Arthur Gevaertlaan 15, 9620 Zottegem"
3867219,165129.96,204954.27,51.154419,4.585008,"Liersesteenweg 87A, 2520 Ranst"
3867220,165129.96,204954.27,51.154419,4.585008,"Liersesteenweg 87A, 2520 Ranst"
3867221,165129.96,204954.27,51.154419,4.585008,"Liersesteenweg 87A, 2520 Ranst"


### Getting The Coordinates Of The Address

Now that we got the belgium address API, we are going to loop over the address to check if the address is there and to get the coordinates. 

In [174]:
for i in range(belgium.shape[0]):
    if address == belgium['address'][i]:
        index = i
        break

In [175]:
index

1213045

In [176]:
belgium.loc[index]

EPSG:31370_x                      103589.46
EPSG:31370_y                      192316.68
EPSG:4326_lat                     51.039167
EPSG:4326_lon                      3.707038
address          Distelstraat 42, 9000 Gent
Name: 1213045, dtype: object

In [177]:
x_value = belgium['EPSG:31370_x'][index]
y_value = belgium['EPSG:31370_y'][index]
latitude = belgium['EPSG:4326_lat'][index]
longitude = belgium['EPSG:4326_lon'][index]

In [263]:
coordinates = {'address' : address,
        'x_value' : x_value, 
        'y_value' : y_value,
        'latitude' : latitude,
        'longitude' : longitude}

In [264]:
coordinates

{'address': 'Distelstraat 42, 9000 Gent',
 'x_value': 103589.46,
 'y_value': 192316.68,
 'latitude': 51.03916669644181,
 'longitude': 3.707038194679286}

### Figure out which tiff file to open depending on the coordinates

To check which tiff file to open, we need to get the boundingbox of the tiff file and then compare if the X and Y of the address is inside the bounding box.

I was able to create a csv file containing the bounding box for all the DSM files. SO we just need to open it and check which tiff file should we use.

In [191]:
bbox = pd.read_csv('data/bounding-box.csv')

In [192]:
bbox.head()

,DSM Files,Left (X),Bottom (Y),Right (X),Top (Y)
0,DSM_1,145000.0,238000.0,162000.0,247000.0
1,DSM_2,162000.0,238000.0,194000.0,250000.0
2,DSM_3,194000.0,238000.0,206000.0,248000.0
3,DSM_4,53000.0,218000.0,66000.0,225500.0
4,DSM_5,66000.0,218000.0,98000.0,232000.0


In [193]:
for i in range(bbox.shape[0]):
    if bbox['Left (X)'][i] <= coordinates['x_value']:
        if bbox['Right (X)'][i] >= coordinates['x_value']:
            if bbox['Bottom (Y)'][i] <= coordinates['y_value']:
                if bbox['Top (Y)'][i] >= coordinates['y_value']:
                    num = i
                    break

In [194]:
bbox.iloc[num]

DSM Files       DSM_22
Left (X)       98000.0
Bottom (Y)    178000.0
Right (X)     130000.0
Top (Y)       198000.0
Name: 21, dtype: object

In [249]:
num = 42

### Downloading the DSM and DTM files

Now that we're able to locate from which tiff file the coordinates is located, we can now start using only this specific tiff files.

In [250]:
# Getting the DSM link
if num < 9:
    DSM = f"https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k0{num+1}.zip"
else:
    DSM = f"https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k{num+1}.zip"


In [251]:
# Getting the DTM link
if num < 9:
    DTM = f"https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dtm-raster-1m/DHMVIIDTMRAS1m_k0{num+1}.zip"
else:
    DTM = f"https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dtm-raster-1m/DHMVIIDTMRAS1m_k{num+1}.zip"

In [258]:
files = {'DSM': DSM, 'DTM': DTM}
files

{'DSM': 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k43.zip',
 'DTM': 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dtm-raster-1m/DHMVIIDTMRAS1m_k43.zip'}

In [268]:
for key, value in files.items():
    with urlopen(value) as zipresp:
        print(f"Downloading the {key} zip file......")
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            print(f"Extracting the {key} zip file .......")
            zfile.extractall(f'data/raster-files/{key}')
            print(f"Done extracting the {key} zip file to raster-files/{key} folder! :)")

Extracting the DSM zip file .......
Done extracting the DSM zip file to raster-files/DSM folder! :)
Extracting the DTM zip file .......
Done extracting the DTM zip file to raster-files/DTM folder! :)


### Creating the Canopy Height Model

Alright! Now the DSM and DTM files are already saved and extracted to the `raster-files` folder.
We can now proceed in creating the <b>Canopy Height Model<b>.

`Canopy Height Model = DSM - DTM`

In [ ]:
from osgeo import gdal 
import matplotlib.pyplot as plt 
  
dataset = gdal.Open(r'data\DSM\DSM_01.tif')

In [ ]:
print(dataset.RasterCount)

In [ ]:
band1 = dataset.GetRasterBand(1)

In [ ]:
b1 = band1.ReadAsArray()

In [ ]:
img = np.dstack((b1)) 
f = plt.figure() 

img.squeeze()

plt.imshow(img)
plt.savefig('Tiff.png') 
plt.show() 

In [ ]:
# Open the file:
dsm = gdal.Open(DSM)
gt = dsm.GetGeoTransorm()
proj = dsm.GetProjection()
band = dsm.GetRasterBand(1)
array = band.ReadAsArray

# Check type of the variable 'raster'
type(raster)